In [43]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import import_gemini
import gemini_analyze_fuction
import gemini_comment_fuction
import gemini_summary_function
import get_and_crawl_data
import plotly_and_upload
import other_source_data

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(import_gemini)
importlib.reload(gemini_analyze_fuction)
importlib.reload(gemini_comment_fuction)
importlib.reload(gemini_summary_function)
importlib.reload(get_and_crawl_data)
importlib.reload(plotly_and_upload)
importlib.reload(other_source_data)

from import_default import *
from import_database import *
from import_other import *
from import_gemini import *
from gemini_analyze_fuction import *
from gemini_comment_fuction import *
from gemini_summary_function import *
from get_and_crawl_data import *
from plotly_and_upload import *
from other_source_data import *

#### Dữ liệu từ Mongo

In [44]:
date_series = get_mongo_collection(ref_db, 'date_series')
time_series = get_mongo_collection(ref_db, 'time_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')
current_quarter_classification_df = get_mongo_collection(ref_db, 'current_quarter_classification')

if date_series['date'].max() < get_today_date():
    new_date = get_today_date()
    date_series = pd.concat([date_series, pd.DataFrame({'date': [new_date]})]).sort_values(by='date', ascending=False).reset_index(drop=True)

In [45]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "pct_change": 1,
    "diff": 1,
    "volume": 1,
    "option": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "RSI_14": 1,
    "week_prev_high": 1,
    "week_prev_low": 1,
    "week_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "month_open": 1,
    "quarter_prev_high": 1,
    "quarter_prev_low": 1,
    "quarter_open": 1,
    "year_open": 1,
    "MFIBO_0382": 1,
    "MFIBO_0500": 1,
    "MFIBO_0618": 1,
    "QFIBO_0382": 1,
    "QFIBO_0500": 1,
    "QFIBO_0618": 1,
	"YFIBO_0382": 1,
    "YFIBO_0500": 1,
    "YFIBO_0618": 1,
    'WPIVOT_P': 1,
    'MPIVOT_P': 1,
}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

In [46]:
today_stock_df = get_mongo_collection(stock_db, "today_stock", projection={"_id": 0, "date": 1, "ticker": 1, "open": 1,"high": 1, "low": 1, "close": 1, "volume": 1, 'pct_change': 1, "t0_score": 1, 'vol_ratio': 1})
today_group_df = get_mongo_collection(stock_db, "today_group")

itd_index_df = get_mongo_collection(stock_db, "itd_index")
itd_stock_df = get_mongo_collection(stock_db, "itd_stock", projection={"_id": 0, "date": 1, "ticker": 1, "close": 1, "t0_score": 1, "vol_ratio": 1})
itd_group_df = get_mongo_collection(stock_db, "itd_group")

other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

#### Tạo các models cho gemini

In [47]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))

# Sắp xếp danh sách model theo thứ tự ổn định
# fast_model_list = select_fast_models(get_gemini_models())
# standard_model_list = select_standard_models(get_gemini_models())
fast_model_list = ['gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']
standard_model_list = ['gemini-2.5-flash', 'gemini-2.0-flash', 'gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

#### Dữ liệu ITD

- Dữ liệu cho VNINDEX

In [48]:
itd_vnindex_df = itd_index_df[itd_index_df['ticker'] == 'VNINDEX'].reset_index(drop=True)
eod_vnindex_df = full_index_df[full_index_df['ticker'] == 'VNINDEX'].reset_index(drop=True)
vnindex_close_prev = eod_vnindex_df[eod_vnindex_df['date'] < itd_vnindex_df['date'].iloc[-1].normalize()].iloc[0]['close'].item()

itd_vnindex_df['cum_volume'] = itd_vnindex_df['volume'][::-1].cumsum()[::-1]
itd_vnindex_df['cum_value'] = eod_vnindex_df['option'].iloc[0]
itd_vnindex_df['pct_change'] = (itd_vnindex_df['close'] - vnindex_close_prev) / vnindex_close_prev
itd_vnindex_df['diff'] = itd_vnindex_df['close'] - vnindex_close_prev

itd_vnindex_df = itd_vnindex_df.set_index('date')

- Dữ liệu cho tâm lý thị trường, biến động giá và độ rộng thanh khoản

In [49]:
itd_stock_dict = {}
for ticker in current_quarter_classification_df['ticker'].unique():
    temp_df = itd_stock_df[itd_stock_df['ticker'] == ticker]
    temp_df['open'] = today_stock_df[today_stock_df['ticker'] == ticker]['open'].item()
    temp_df['diff'] = temp_df['close'] - temp_df['open']
    itd_stock_dict[ticker] = temp_df

temp_sentiment_df = time_series.set_index('date')
temp_diff_df = time_series.set_index('date')
temp_vol_ratio_df = time_series.set_index('date')
for ticker, df in itd_stock_dict.items():
    temp_df = df.dropna().set_index('date')
    temp_sentiment_df[ticker] = temp_df['t0_score']
    temp_diff_df[ticker] = temp_df['diff']
    temp_vol_ratio_df[ticker] = temp_df['vol_ratio']

sentiment_stock_df = time_series.set_index('date')
sentiment_stock_df['sentiment_pos'] = (temp_sentiment_df > 0.2).sum(axis=1) / len(current_quarter_classification_df)
sentiment_stock_df['sentiment_neg'] = (temp_sentiment_df <= -1).sum(axis=1) / len(current_quarter_classification_df)

diff_stock_df = time_series.set_index('date')
diff_stock_df['price_pos'] = (temp_diff_df > 0).sum(axis=1)
diff_stock_df['price_neu'] = (temp_diff_df == 0).sum(axis=1)
diff_stock_df['price_neg'] = (temp_diff_df < 0).sum(axis=1)

vol_ratio_stock_df = time_series.set_index('date')
vol_ratio_stock_df['vol_ratio_pos'] = (temp_vol_ratio_df > 1.0).sum(axis=1)
vol_ratio_stock_df['vol_ratio_neu'] = ((temp_vol_ratio_df > 0.7) & (temp_vol_ratio_df <= 1.0)).sum(axis=1)
vol_ratio_stock_df['vol_ratio_neg'] = (temp_vol_ratio_df <= 0.7).sum(axis=1)

sentiment_stock_df = sentiment_stock_df.dropna()
diff_stock_df = diff_stock_df.dropna()
vol_ratio_stock_df = vol_ratio_stock_df.dropna()

- Tổng hợp và ghép thêm giá trị thanh khoản

In [50]:
daily_15h00_itd_df = time_series.set_index('date')
daily_15h00_itd_df = pd.concat([daily_15h00_itd_df, itd_vnindex_df], axis=1)
daily_15h00_itd_df = pd.concat([daily_15h00_itd_df, sentiment_stock_df], axis=1)
daily_15h00_itd_df = pd.concat([daily_15h00_itd_df, diff_stock_df], axis=1)
daily_15h00_itd_df = pd.concat([daily_15h00_itd_df, vol_ratio_stock_df], axis=1)
daily_15h00_itd_df = pd.concat([daily_15h00_itd_df, itd_group_df[itd_group_df['ticker'] == 'all'].set_index('date')['vol_ratio']], axis=1)
daily_15h00_itd_df = daily_15h00_itd_df.sort_index().dropna().reset_index()

- Nhận xét biến động VNINDEX

In [51]:
def daily_vn30f1m_overall_comment(df):
    prompt = f"""
    Đây là dữ liệu của chỉ số VNINDEX:
    {df.to_csv(index=False, sep='|', lineterminator='\n')}

    **Bối cảnh:**
    Hãy đóng vai là một nhà phân tích tài chính chuyên nghiệp và khách quan của một công ty chứng khoán. Nhiệm vụ của bạn là phân tích dữ liệu giá theo phút của chỉ số VNINDEX được cung cấp và viết một báo cáo ngắn gọn, súc tích về diễn biến của phiên giao dịch buổi.

    **Dữ liệu đầu vào:**
    Dữ liệu là một chuỗi văn bản chứa thông tin giao dịch theo từng phút, bao gồm các cột: `date`, `ticker`, `open`, `high`, `low`, `close`.

    **Yêu cầu đầu ra:**
    Kết quả trả về phải tuân thủ nghiêm ngặt các quy tắc sau:

    * **Định dạng:** Chỉ trả về **duy nhất** đoạn văn phân tích. **Tuyệt đối không** thêm bất kỳ câu dẫn dắt, tiêu đề, hay lời chào nào (ví dụ: không được viết "Dưới đây là báo cáo..." hoặc "Sau khi phân tích...").
    * **Cấu trúc:** Đoạn văn phải có chính xác **3 câu**. Mỗi câu từ 25 đến 30 từ.
    * **Giọng điệu:** Chuyên nghiệp, khách quan, mang tính tường thuật, không đưa ra khuyến nghị mua/bán.
    * **Nguồn:** Mọi phân tích phải dựa hoàn toàn vào dữ liệu được cung cấp.
    * **LUÔN LUÔN** sử dụng ngày dạng "dd/mm" trong bài viết khi được yêu cầu.

    **Chi tiết hướng dẫn cho từng câu:**

    * **Câu 1: Bối cảnh và Diễn biến mở đầu (khoảng 15-20 từ)**
        * **Mục đích:** Giới thiệu bối cảnh phiên giao dịch và mô tả xu hướng ngay sau khi thị trường mở cửa.
        * **Hướng dẫn nội dung:**
            1.  Bắt đầu câu bằng cụm từ "Tổng kết phiên giao dịch ngày `[DD/MM/YYYY]`...". Bạn phải tự động trích xuất và định dạng ngày từ cột `date` trong dữ liệu.
            2.  Nhận xét ngắn gọn về trạng thái mở cửa (ví dụ: "mở cửa khá ổn định", "mở cửa trong sắc xanh", "chịu áp lực ngay từ đầu").
            3.  Mô tả hành động giá ngay sau đó (ví dụ: "nhưng nhanh chóng chịu áp lực bán", "tuy nhiên đà tăng không duy trì được", "và tiếp tục nới rộng đà giảm").

    * **Câu 2: Diễn biến chính và Mức biến động (khoảng 15-20 từ)**
        * **Mục đích:** Tóm tắt xu hướng chủ đạo trong phiên và định lượng mức độ biến động mạnh nhất.
        * **Hướng dẫn nội dung:**
            1.  Mô tả xu hướng chính kéo dài trong phiên (ví dụ: "Xu hướng tiêu cực này chiếm ưu thế", "Lực mua chủ động xuyên suốt phiên").
            2.  Làm nổi bật mức biến động lớn nhất bằng cách tính chênh lệch giữa giá trị `high` cao nhất và `low` thấp nhất trong toàn bộ dữ liệu. Diễn đạt nó dưới dạng "đẩy chỉ số có thời điểm giảm hơn `X` điểm so với mức cao nhất" hoặc "giúp chỉ số có lúc tăng hơn `X` điểm so với mức điểm thấp nhất" .

    * **Câu 3: Tổng kết cuối phiên (khoảng 15-20 từ)**
        * **Mục đích:** Mô tả các nỗ lực vào cuối phiên và chốt lại trạng thái của giá đóng cửa.
        * **Hướng dẫn nội dung:**
            1.  Nhận xét về các diễn biến trong giai đoạn cuối phiên (ví dụ: "Mặc dù một nỗ lực phục hồi...", "Áp lực bán gia tăng trở lại...", "Xu hướng vẫn tiếp diễn...").
            2.  Đưa ra kết luận về giá đóng cửa so với mặt bằng chung của phiên (ví dụ: "chỉ số vẫn duy trì ở vùng giá thấp", "bảo toàn được thành quả", "thu hẹp đáng kể đà giảm").
    """
    return prompt

promt_df = daily_15h00_itd_df[['date', 'ticker', 'open', 'high', 'low', 'close', 'volume']]
overall_vnindex_comment = generate_content_with_model_dict(standard_model_dict, daily_vn30f1m_overall_comment(promt_df), 'daily_vnindex_overall_comment')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vnindex_overall_comment.


#### Dữ liệu khối ngoại trong phiên

- Lịch sử mua bán khối ngoại

In [52]:
#Dữ liệu NN từ Fireant
daily_15h00_nn_data_df = nntd_index_df[(nntd_index_df['ticker'].isin(['HNXINDEX', 'UPINDEX', 'VNINDEX'])) & (nntd_index_df['type'] == 'NN')].reset_index(drop=True)
daily_15h00_nn_data_df = daily_15h00_nn_data_df.groupby(['date']).sum().iloc[:, :6].sort_values('date', ascending = False).reset_index()

- Top cổ phiếu mua bán trong phiên

In [53]:
type_df = pd.DataFrame()
for ticker in nntd_stock_df['ticker'].unique():
    temp_df = nntd_stock_df[(nntd_stock_df['ticker'] == ticker) & (nntd_stock_df['type'] == "NN")]
    temp_df = temp_df.sort_values('date', ascending=False).reset_index(drop=True)
    type_df = pd.concat([type_df, temp_df[['type', 'ticker', 'net_value']].iloc[[0]]], ignore_index=True).sort_values('net_value', ascending=False)

# Gán kết quả và tổng hợp
daily_15h00_nn_stock_df = pd.concat([type_df.head(10), type_df.tail(10)], ignore_index=True)
daily_15h00_nn_stock_df['top_check'] = (daily_15h00_nn_stock_df['net_value'] > 0).astype(int)
daily_15h00_nn_stock_df['net_value'] = abs(daily_15h00_nn_stock_df['net_value'])

#### Dòng tiền và thanh khoản các nhóm ngành

In [54]:
daily_15h00_industry_df = today_group_df[~today_group_df['ticker'].isin(['all', 'hsA', 'hsB', 'hsC', 'hsD', 'large', 'mid', 'small', 'penny'])]
daily_15h00_industry_df = daily_15h00_industry_df[['ticker', 'vol_ratio', 't0_score']].sort_values('t0_score', ascending=False).reset_index(drop=True)
daily_15h00_industry_df['ticker'] = daily_15h00_industry_df['ticker'].map(name_map_dict)

#### Top cổ phiếu

In [55]:
top_stock_df = today_stock_df[today_stock_df['ticker'].isin(current_quarter_classification_df['ticker'].unique())][['ticker', 'close', 'pct_change', 't0_score', 'vol_ratio', 'volume']]
top_stock_df['value'] = top_stock_df['close'] * top_stock_df['volume']
top_stock_df = top_stock_df[(top_stock_df['value'] > 50000000) & (top_stock_df['volume'] > 500000)].sort_values('t0_score', ascending=False)
top_stock_df['industry'] = top_stock_df['ticker'].map(full_stock_classification_df.set_index('ticker')['industry_name'])
top_stock_df['industry'] = top_stock_df['industry'].map(name_map_dict)

top_stock_pos = top_stock_df.head(10)
top_stock_pos['type'] = 'pos'
top_stock_neg = top_stock_df.tail(10).sort_values('t0_score')
top_stock_neg['type'] = 'neg'

daily_15h00_top_stock_df = pd.concat([top_stock_pos, top_stock_neg], ignore_index=True)
daily_15h00_top_stock_df = daily_15h00_top_stock_df[['ticker', 'industry', 'close', 'pct_change', 't0_score', 'vol_ratio', 'type']]

#### Lưu vào MSSQL

In [56]:
daily_15h00_comment_df = pd.DataFrame(
    {
        "type":     ["overall", "date", "time"],
        "comment": [
            overall_vnindex_comment,
            time_series.iloc[0].item().strftime('%d/%m/%Y'),
            time_series.iloc[0].item().strftime('%H:%M')
        ],
    }
)

In [57]:
%%capture
save_to_mssql(cts_engine, daily_15h00_comment_df, 'daily_15h00_comment')
save_to_mssql(cts_engine, daily_15h00_itd_df, 'daily_15h00_itd')
save_to_mssql(cts_engine, daily_15h00_nn_data_df, 'daily_15h00_nn_data')
save_to_mssql(cts_engine, daily_15h00_nn_stock_df, 'daily_15h00_nn_stock')
save_to_mssql(cts_engine, daily_15h00_industry_df, 'daily_15h00_industry')
save_to_mssql(cts_engine, daily_15h00_top_stock_df, 'daily_15h00_top_stock')
